In [25]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import utils as np_utils
from keras import preprocessing

In [26]:
train_data_folder='Dataset/train/'
test_data_folder='Dataset/test/'
train_files=os.listdir(train_data_folder)

In [27]:
complete_data=None
string_to_int={}
int_to_string={}
counter=0

def fill_data(filename):
    data=np.load(file=train_data_folder+filename)
    filename=filename.split('.')[0]
    temp_var=-1
    
    global counter
    global complete_data
    global string_to_int
    global int_to_string
    
    if filename in string_to_int:
        temp_var=string_to_int[filename]
    
    else:
        temp_var=counter
        counter+=1
        string_to_int[filename]=temp_var
        int_to_string[temp_var]=filename
    
    dimension=data.shape
    temp_y=np.ones(shape=(dimension[0],1),dtype=np.float64)*temp_var
    data=np.concatenate((data,temp_y),axis=1)
    
    if complete_data is None:
        complete_data=np.copy(data)
    
    else:
        complete_data=np.concatenate((complete_data,data),axis=0)
    

for filename in train_files:
    fill_data(filename)

print(complete_data.shape)
print(string_to_int)
print(int_to_string)    

(100000, 785)
{'penguin': 13, 'keyboard': 3, 'flashlight': 9, 'trombone': 16, 'skyscraper': 0, 'nose': 1, 'parrot': 8, 'bulldozer': 12, 'spider': 11, 'snowman': 10, 'hat': 4, 'eyeglasses': 5, 'violin': 7, 'hand': 17, 'pig': 15, 'chair': 6, 'harp': 14, 'laptop': 18, 'banana': 2, 'foot': 19}
{0: 'skyscraper', 1: 'nose', 2: 'banana', 3: 'keyboard', 4: 'hat', 5: 'eyeglasses', 6: 'chair', 7: 'violin', 8: 'parrot', 9: 'flashlight', 10: 'snowman', 11: 'spider', 12: 'bulldozer', 13: 'penguin', 14: 'harp', 15: 'pig', 16: 'trombone', 17: 'hand', 18: 'laptop', 19: 'foot'}


In [28]:
complete_data=np.random.permutation(complete_data)
print(complete_data.shape)

(100000, 785)


In [29]:
[instances,dimensions]=complete_data.shape

x_train=complete_data[:,0:dimensions-1].astype(np.float32)
test_x=np.load(file="Dataset/test/test.npy").astype(np.float32)
y_train2=complete_data[:,dimensions-1:].astype(np.int32)
y_train = np_utils.to_categorical(y_train2)
print(x_train.shape)
print(y_train.shape)
print(y_train2.shape)
print(test_x.shape)
categories=20

(100000, 784)
(100000, 20)
(100000, 1)
(100000, 784)


In [30]:
# param=1000
# x_train=x_train[0:param,:]
# y_train=y_train[0:param]
# y_train.shape
x_train/=255.0
test_x/=255.0
x_train=x_train.reshape((100000,28,28,1))
test_x=test_x.reshape((100000,28,28,1))

In [31]:
model = Sequential()
model.add(Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(Dropout(0.35))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.45))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(categories, activation='softmax'))

In [32]:
datagen = preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [33]:
testI=int(instances*0.8)
x_test=x_train[testI:,:]
y_test=y_train[testI:,:]
x_train=x_train[:testI,:]
y_train=y_train[:testI,:]

In [ ]:
epochs = 500
batch_size=2048

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,verbose=1,validation_data=(x_test,y_test))

Epoch 1/500


/home/ankesh/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/ankesh/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


40/40 [==============================] - 14s 358ms/step - loss: 0.2837 - acc: 0.9188 - val_loss: 0.2736 - val_acc: 0.9239
Epoch 2/500
40/40 [==============================] - 13s 316ms/step - loss: 0.2782 - acc: 0.9195 - val_loss: 0.3989 - val_acc: 0.8949
Epoch 3/500
40/40 [==============================] - 13s 324ms/step - loss: 0.2744 - acc: 0.9201 - val_loss: 0.6289 - val_acc: 0.8587
Epoch 4/500
40/40 [==============================] - 13s 321ms/step - loss: 0.2690 - acc: 0.9220 - val_loss: 0.5049 - val_acc: 0.8795
Epoch 5/500
40/40 [==============================] - 13s 327ms/step - loss: 0.2698 - acc: 0.9224 - val_loss: 0.3017 - val_acc: 0.9200
Epoch 6/500
40/40 [==============================] - 13s 324ms/step - loss: 0.2724 - acc: 0.9209 - val_loss: 0.3707 - val_acc: 0.9037
Epoch 7/500
40/40 [==============================] - 13s 326ms/step - loss: 0.2719 - acc: 0.9224 - val_loss: 0.4303 - val_acc: 0.8936
Epoch 8/500
40/40 [==============================] - 13s 329ms/step - loss

Epoch 62/500
40/40 [==============================] - 13s 329ms/step - loss: 0.2637 - acc: 0.9232 - val_loss: 0.4204 - val_acc: 0.8975
Epoch 63/500
40/40 [==============================] - 13s 319ms/step - loss: 0.2596 - acc: 0.9231 - val_loss: 0.4649 - val_acc: 0.8864
Epoch 64/500
40/40 [==============================] - 13s 323ms/step - loss: 0.2583 - acc: 0.9240 - val_loss: 0.4102 - val_acc: 0.8984
Epoch 65/500
40/40 [==============================] - 13s 324ms/step - loss: 0.2656 - acc: 0.9228 - val_loss: 0.3736 - val_acc: 0.9047
Epoch 66/500
40/40 [==============================] - 13s 325ms/step - loss: 0.2602 - acc: 0.9236 - val_loss: 0.5709 - val_acc: 0.8686
Epoch 67/500
40/40 [==============================] - 13s 326ms/step - loss: 0.2611 - acc: 0.9234 - val_loss: 0.6629 - val_acc: 0.8520
Epoch 68/500
40/40 [==============================] - 13s 327ms/step - loss: 0.2604 - acc: 0.9236 - val_loss: 0.2726 - val_acc: 0.9247
Epoch 69/500
40/40 [==============================] - 1

In [11]:
# batch_size = 2048
# epochs = 500
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.1)
# # model1.evaluate(test_data, test_labels_one_hot)

In [23]:
def label_to_string(label):
    lis=[]
    instances=label.shape[0]
    for i in range(instances):
        lis.append(int_to_string[label[i]])
    return lis

In [21]:
model.save_weights("big2.h5")

In [24]:
predictions=np.argmax(model.predict(test_x),axis=1)
print(predictions.shape)
predictions=label_to_string(predictions)
predictions=np.asarray(predictions)
print(predictions.shape)

(100000,)
(100000,)


In [25]:
df=pd.DataFrame(data=predictions,columns=['CATEGORY'])
df.index.name='ID'
df.to_csv(path_or_buf="ignore/CNN_sub.csv",sep=',')

In [16]:

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")
 
# # later...
 
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [9]:
model.load_weights("big.h5")